In [1]:
# DONE

# FT-HMC implemented for 8x8 2D QED (using SiLU as activation function).

# Try to minimize size of the force in training. No significant improvements.

# Some test on ergodicity
# (calculate the probablity of generating the configs obtained via conventional HMC).

# TODO

# Plot the force size distribution
# Is the large force from the original action or Field-Transformation the determinant?
# If from the determinant, then the fermion force won't cause problem for HMC

# Use the same Field-Transformation for larger system (say 16x16, 32x32, 64x64, etc)
# Study how the delta H depends on the system size ( perhaps delta H ~ sqrt(volume) )

# Study the auto-correlation for observables, topo, plaq, flowed plaq, etc.

# Improving the Field-Transformation to reduce force.

In [19]:
import torch
import math
import sys
import os
import numpy as np
from timeit import default_timer as timer
from functools import reduce
from field_transformation import *

In [113]:
# From Xiao-Yong

class Param:
    def __init__(self, beta = 6.0, lat = [64, 64], tau = 2.0, nstep = 50, ntraj = 256, nrun = 4, nprint = 256, seed = 11*13, randinit = False, nth = int(os.environ.get('OMP_NUM_THREADS', '2')), nth_interop = 2):
        self.beta = beta
        self.lat = lat
        self.nd = len(lat)
        self.volume = reduce(lambda x,y:x*y, lat)
        self.tau = tau
        self.nstep = nstep
        self.dt = self.tau / self.nstep
        self.ntraj = ntraj
        self.nrun = nrun
        self.nprint = nprint
        self.seed = seed
        self.randinit = randinit
        self.nth = nth
        self.nth_interop = nth_interop
    def initializer(self):
        if self.randinit:
            return torch.empty((self.nd,) + self.lat).uniform_(-math.pi, math.pi)
        else:
            return torch.zeros((self.nd,) + self.lat)
    def summary(self):
        return f"""latsize = {self.lat}
volume = {self.volume}
beta = {self.beta}
trajs = {self.ntraj}
tau = {self.tau}
steps = {self.nstep}
seed = {self.seed}
nth = {self.nth}
nth_interop = {self.nth_interop}
"""
    def uniquestr(self):
        lat = ".".join(str(x) for x in self.lat)
        return f"out_l{lat}_b{self.beta}_n{self.ntraj}_t{self.tau}_s{self.nstep}.out"

def action(param, f):
    return (-param.beta)*torch.sum(torch.cos(plaqphase(f)))

def force(param, f):
    f.requires_grad_(True)
    s = action(param, f)
    f.grad = None
    s.backward()
    ff = f.grad
    f.requires_grad_(False)
    return ff

plaqphase = lambda f: f[0,:] - f[1,:] - torch.roll(f[0,:], shifts=-1, dims=1) + torch.roll(f[1,:], shifts=-1, dims=0)
topocharge = lambda f: torch.floor(0.1 + torch.sum(regularize(plaqphase(f))) / (2*math.pi))
def regularize(f):
    p2 = 2*math.pi
    f_ = (f - math.pi) / p2
    return p2*(f_ - torch.floor(f_) - 0.5)

def leapfrog(param, x, p):
    dt = param.dt
    x_ = x + 0.5*dt*p
    f = force(param, x_)
    p_ = p + (-dt)*f
    print(f'plaq(x) {action(param, x) / (-param.beta*param.volume)}  force.norm {torch.linalg.norm(f)}')
    for i in range(param.nstep-1):
        x_ = x_ + dt*p_
        p_ = p_ + (-dt)*force(param, x_)
    x_ = x_ + 0.5*dt*p_
    return (x_, p_)
def hmc(param, x):
    p = torch.randn_like(x)
    act0 = action(param, x) + 0.5*torch.sum(p*p)
    x_, p_ = leapfrog(param, x, p)
    xr = regularize(x_)
    act = action(param, xr) + 0.5*torch.sum(p_*p_)
    prob = torch.rand([], dtype=torch.float64)
    dH = act-act0
    exp_mdH = torch.exp(-dH)
    acc = prob < exp_mdH
    newx = xr if acc else x
    return (dH, exp_mdH, acc, newx)

put = lambda s: sys.stdout.write(s)

In [5]:
param = Param(
    beta = 2.0,
    lat = (8, 8),
    tau = 2, # 0.3
    nstep = 8, # 3
    # ADJUST ME
    ntraj = 2, # 2**16 # 2**10 # 2**15
    #
    nprint = 2,
    seed = 1331)

In [6]:
torch.manual_seed(param.seed)

torch.set_num_threads(param.nth)
torch.set_num_interop_threads(param.nth_interop)
os.environ["OMP_NUM_THREADS"] = str(param.nth)
os.environ["KMP_BLOCKTIME"] = "0"
os.environ["KMP_SETTINGS"] = "1"
os.environ["KMP_AFFINITY"]= "granularity=fine,verbose,compact,1,0"

torch.set_default_tensor_type(torch.DoubleTensor)

In [104]:
def run(param, field = None):
    if field is None:
        field = param.initializer()
    with open(param.uniquestr(), "w") as O:
        params = param.summary()
        O.write(params)
        put(params)
        plaq, topo = (action(param, field) / (-param.beta*param.volume), topocharge(field))
        status = f"Initial configuration:  plaq: {plaq}  topo: {topo} {field.shape}\n"
        O.write(status)
        put(status)
        ts = []
        for n in range(param.nrun):
            t = -timer()
            for i in range(param.ntraj):
                dH, exp_mdH, acc, field = hmc(param, field)
                plaq = action(param, field) / (-param.beta*param.volume)
                topo = topocharge(field)
                ifacc = "ACCEPT" if acc else "REJECT"
                status = f"Traj: {n*param.ntraj+i+1:4}  {ifacc}:  dH: {dH:< 12.8}  exp(-dH): {exp_mdH:< 12.8}  plaq: {plaq:< 12.8}  topo: {topo:< 3.3}\n"
                O.write(status)
                if (i+1) % (param.ntraj//param.nprint) == 0:
                    put(status)
            t += timer()
            ts.append(t)
        print("Run times: ", ts)
        print("Per trajectory: ", [t/param.ntraj for t in ts])
    return field
field = run(param)

latsize = (8, 8)
volume = 64
beta = 2.0
trajs = 4
tau = 0.5
steps = 64
seed = 1331
nth = 2
nth_interop = 2
Initial configuration:  plaq: 1.0  topo: 0.0 torch.Size([2, 8, 8])
plaq(x) 1.0  force.norm 0.28484840702762815
Traj:    1  ACCEPT:  dH: -0.0030182743  exp(-dH):  1.0030228    plaq:  0.79750934   topo:  0.0
plaq(x) 0.7975093376581902  force.norm 18.620514556154312
Traj:    2  ACCEPT:  dH: -0.00058593622  exp(-dH):  1.0005861    plaq:  0.69746949   topo:  0.0
plaq(x) 0.6974694869063531  force.norm 19.25952813968039
Traj:    3  ACCEPT:  dH:  0.00035171744  exp(-dH):  0.99964834   plaq:  0.69743842   topo:  0.0
plaq(x) 0.69743841846725  force.norm 18.004364386744186
Traj:    4  ACCEPT:  dH:  0.00058309356  exp(-dH):  0.99941708   plaq:  0.78234744   topo:  0.0
plaq(x) 0.7823474412175998  force.norm 16.620225849561447
Traj:    5  ACCEPT:  dH: -0.00069129855  exp(-dH):  1.0006915    plaq:  0.7515997    topo:  0.0
plaq(x) 0.7515996999323443  force.norm 18.63207422212415
Traj:    6  ACCEP

In [8]:
def ft_flow(flow, f):
    for layer in flow:
        f, lJ = layer.forward(f)
    return f.detach()

def ft_flow_inv(flow, f):
    for layer in reversed(flow):
        f, lJ = layer.reverse(f)
    return f.detach()

def ft_action(param, flow, f):
    y = f
    logJy = 0.0
    for layer in flow:
        y, lJ = layer.forward(y)
        logJy += lJ
    action = U1GaugeAction(param.beta)
    s = action(y) - logJy
    return s

def ft_force(param, flow, field, create_graph = False):
    # f is the field follows the transformed distribution (close to prior distribution)
    f = field
    f.requires_grad_(True)
    s = ft_action(param, flow, f)
    ss = torch.sum(s)
    # f.grad = None
    ff, = torch.autograd.grad(ss, f, create_graph = create_graph)
    f.requires_grad_(False)
    return ff

In [9]:
def train_step(model, action, optimizer, metrics, batch_size, with_force = False, pre_model = None):
    layers, prior = model['layers'], model['prior']
    optimizer.zero_grad()
    #
    xi = None
    if pre_model != None:
        pre_layers, pre_prior = pre_model['layers'], pre_model['prior']
        pre_xi = pre_prior.sample_n(batch_size)
        x = ft_flow(pre_layers, pre_xi)
        xi = ft_flow_inv(layers, x)
    #
    xi, x, logq = apply_flow_to_prior(prior, layers, batch_size=batch_size, xi=xi)
    logp = -action(x)
    #
    force_size = torch.tensor(0.0)
    dkl = calc_dkl(logp, logq)
    loss = torch.tensor(0.0)
    if with_force:
        assert pre_model != None
        force = ft_force(param, layers, xi, True)
        force_size = torch.sum(torch.square(force))
        loss = force_size
    else:
        loss = dkl
    #
    loss.backward()
    #
    # minimization target
    # loss mini
    # -> (logq - logp) mini
    # -> (action - logJ) mini
    #
    optimizer.step()
    ess = compute_ess(logp, logq)
    #
    print(grab(loss),
          grab(force_size),
          grab(dkl),
          grab(ess),
          torch.linalg.norm(ft_force(param, layers, xi)))
    #
    metrics['loss'].append(grab(loss))
    metrics['force'].append(grab(force_size))
    metrics['dkl'].append(grab(dkl))
    metrics['logp'].append(grab(logp))
    metrics['logq'].append(grab(logq))
    metrics['ess'].append(grab(ess))

def flow_train(param, with_force = False, pre_model = None):  # packaged from original ipynb by Xiao-Yong Jin
    # Theory
    lattice_shape = param.lat
    link_shape = (2,*param.lat)
    beta = param.beta
    u1_action = U1GaugeAction(beta)
    # Model
    prior = MultivariateUniform(torch.zeros(link_shape), 2*np.pi*torch.ones(link_shape))
    #
    n_layers = 24
    n_s_nets = 2
    hidden_sizes = [8,8]
    kernel_size = 3
    layers = make_u1_equiv_layers(lattice_shape=lattice_shape, n_layers=n_layers, n_mixture_comps=n_s_nets,
                                  hidden_sizes=hidden_sizes, kernel_size=kernel_size)
    set_weights(layers)
    model = {'layers': layers, 'prior': prior}
    # Training
    base_lr = .001
    optimizer = torch.optim.Adam(model['layers'].parameters(), lr=base_lr)
    optimizer_wf = torch.optim.Adam(model['layers'].parameters(), lr=base_lr / 100.0)
    #
    # ADJUST ME
    N_era = 10
    N_epoch = 100
    #
    batch_size = 64
    print_freq = N_epoch # epochs
    plot_freq = 1 # epochs
    history = {
        'loss' : [],
        'force' : [],
        'dkl' : [],
        'logp' : [],
        'logq' : [],
        'ess' : []
    }
    for era in range(N_era):
        for epoch in range(N_epoch):
            train_step(model, u1_action, optimizer, history, batch_size)
            if with_force:
                train_step(model, u1_action, optimizer_wf, history, batch_size,
                           with_force = with_force, pre_model = pre_model)
            if epoch % print_freq == 0:
                print_metrics(history, print_freq, era, epoch)
    return model,u1_action

def flow_eval(model, u1_action):  # packaged from original ipynb by Xiao-Yong Jin
    ensemble_size = 1024
    u1_ens = make_mcmc_ensemble(model, u1_action, 64, ensemble_size)
    print("Accept rate:", np.mean(u1_ens['accepted']))
    Q = grab(topo_charge(torch.stack(u1_ens['x'], axis=0)))
    X_mean, X_err = bootstrap(Q**2, Nboot=100, binsize=16)
    print(f'Topological susceptibility = {X_mean:.2f} +/- {X_err:.2f}')
    print(f'... vs HMC estimate = 1.23 +/- 0.02')

In [10]:
pre_flow_model, flow_act = flow_train(param)
flow_eval(pre_flow_model,flow_act)
pre_flow = pre_flow_model['layers']

-235.80379178382825 0.0 -235.80379178382825 0.015742986900554163 tensor(173.9276)
== Era 0 | Epoch 0 metrics ==
	loss -235.804
	force 0
	dkl -235.804
	logp 0.693854
	logq -235.11
	ess 0.015743
-240.22592204900428 0.0 -240.22592204900428 0.01726504117436026 tensor(169.3064)
-241.73488535507389 0.0 -241.73488535507389 0.016400911777968756 tensor(167.0606)
-247.4616937980544 0.0 -247.4616937980544 0.015810153849540865 tensor(160.3539)
-253.25240939507333 0.0 -253.25240939507333 0.01941417680419557 tensor(158.7390)
-256.75166216575127 0.0 -256.75166216575127 0.015804690643426364 tensor(154.5671)
-257.21331835493976 0.0 -257.21331835493976 0.01562887239859159 tensor(157.1432)
-261.5854650563474 0.0 -261.5854650563474 0.020297799438260253 tensor(151.7423)
-265.36695333901866 0.0 -265.36695333901866 0.0156412599546812 tensor(151.7542)
-266.2862499543668 0.0 -266.2862499543668 0.016146700456890064 tensor(155.4196)
-270.4000967148002 0.0 -270.4000967148002 0.01831122343147412 tensor(156.1291)
-

-282.3031800495452 0.0 -282.3031800495452 0.03519414026309184 tensor(173.6366)
-281.7106893399856 0.0 -281.7106893399856 0.044429059801847706 tensor(189.9906)
-282.02463668393204 0.0 -282.02463668393204 0.02869624631395041 tensor(171.9115)
-283.132869239756 0.0 -283.132869239756 0.019255968703855045 tensor(165.2447)
-282.06928167181843 0.0 -282.06928167181843 0.02305745703770662 tensor(170.5575)
-282.63741013978205 0.0 -282.63741013978205 0.03463303707863622 tensor(174.0301)
-282.69424815211875 0.0 -282.69424815211875 0.07775488816439124 tensor(182.1365)
-282.83367070770777 0.0 -282.83367070770777 0.1173986095789971 tensor(173.4950)
-282.4292215109003 0.0 -282.4292215109003 0.04353524233108277 tensor(174.8651)
-282.63920432682426 0.0 -282.63920432682426 0.057467884980354454 tensor(174.7497)
-282.52266144131096 0.0 -282.52266144131096 0.1869519973917699 tensor(175.9306)
-282.3960594537949 0.0 -282.3960594537949 0.10639430865819953 tensor(175.4130)
-283.1874588939615 0.0 -283.18745889396

-285.47454534289994 0.0 -285.47454534289994 0.08551370972478504 tensor(159.2134)
-285.7410910826486 0.0 -285.7410910826486 0.09694266245843546 tensor(166.2664)
-285.1816577242462 0.0 -285.1816577242462 0.18581720592138307 tensor(160.6161)
-284.9456329769272 0.0 -284.9456329769272 0.037887675923887025 tensor(165.5799)
-285.1367904117073 0.0 -285.1367904117073 0.0709500809313619 tensor(161.8093)
-284.8899231454957 0.0 -284.8899231454957 0.1292122816474404 tensor(175.4481)
-285.48417859246416 0.0 -285.48417859246416 0.09883028991632839 tensor(170.3434)
-285.20711737021287 0.0 -285.20711737021287 0.1686882618690036 tensor(159.7044)
-285.5761087567976 0.0 -285.5761087567976 0.10562976216060008 tensor(153.3455)
-284.9573830256865 0.0 -284.9573830256865 0.1613954717331739 tensor(166.5366)
-285.52831036667357 0.0 -285.52831036667357 0.06433101101355754 tensor(157.1581)
-285.5380265392524 0.0 -285.5380265392524 0.08475295738353089 tensor(168.5715)
-285.52151778254535 0.0 -285.52151778254535 0.0

-285.2153668369717 0.0 -285.2153668369717 0.2189449581050638 tensor(171.8966)
-285.48660306425217 0.0 -285.48660306425217 0.11321807829975229 tensor(187.9403)
-285.5833949046988 0.0 -285.5833949046988 0.0637315382550267 tensor(158.9213)
-285.48727051687985 0.0 -285.48727051687985 0.0839953258493482 tensor(182.5070)
-285.54272284288584 0.0 -285.54272284288584 0.049358031771688855 tensor(175.0742)
-285.1236794662193 0.0 -285.1236794662193 0.17802011448794844 tensor(176.3286)
-285.8453044992894 0.0 -285.8453044992894 0.1956267839023526 tensor(155.9885)
-286.1826945515369 0.0 -286.1826945515369 0.20304134682897104 tensor(165.7703)
-285.6863122326545 0.0 -285.6863122326545 0.2116240683831535 tensor(173.1717)
-285.771364346384 0.0 -285.771364346384 0.23513026087984623 tensor(168.5351)
-285.317134575164 0.0 -285.317134575164 0.09205607013542556 tensor(185.0386)
-285.3653766845782 0.0 -285.3653766845782 0.07107531753232464 tensor(166.2538)
-285.29057236143143 0.0 -285.29057236143143 0.15304830

-286.22982986846034 0.0 -286.22982986846034 0.20246256551850836 tensor(171.2551)
-286.1820292900853 0.0 -286.1820292900853 0.15544772488880687 tensor(171.7185)
-286.3019592642024 0.0 -286.3019592642024 0.2082460121477486 tensor(164.9257)
-286.8255649181763 0.0 -286.8255649181763 0.29026847375336307 tensor(173.1734)
-286.4177581006928 0.0 -286.4177581006928 0.1026103578595871 tensor(182.5192)
-286.35691993734576 0.0 -286.35691993734576 0.05777089000652167 tensor(164.2266)
-286.1784712944655 0.0 -286.1784712944655 0.09900306937663494 tensor(172.7846)
-286.43381642512065 0.0 -286.43381642512065 0.09164166897145622 tensor(175.3758)
-286.512558083341 0.0 -286.512558083341 0.08394243379631174 tensor(184.6040)
-286.49338604286334 0.0 -286.49338604286334 0.1519100506888046 tensor(171.0245)
-286.11971537533964 0.0 -286.11971537533964 0.09586047127971038 tensor(221.3763)
-286.27289834545763 0.0 -286.27289834545763 0.14550963010592424 tensor(202.0609)
-286.6831738102135 0.0 -286.6831738102135 0.1

-286.5381218721758 0.0 -286.5381218721758 0.21287292229146673 tensor(259.0772)
-286.10304876748773 0.0 -286.10304876748773 0.14757232188692027 tensor(177.4414)
-286.7988142692842 0.0 -286.7988142692842 0.1992576221148997 tensor(175.7745)
-286.88955565189383 0.0 -286.88955565189383 0.2066341960387955 tensor(212.8828)
-286.73436938298346 0.0 -286.73436938298346 0.23961185995327375 tensor(198.8368)
-286.3219745816996 0.0 -286.3219745816996 0.14143779851890073 tensor(222.2222)
-286.25043914365307 0.0 -286.25043914365307 0.14604459521002403 tensor(155.7067)
-286.80813812640616 0.0 -286.80813812640616 0.25228464373634757 tensor(207.6366)
-286.7118538872136 0.0 -286.7118538872136 0.10008081764022479 tensor(187.6564)
-286.7248982664024 0.0 -286.7248982664024 0.24895956847001513 tensor(175.3379)
-286.2574371719129 0.0 -286.2574371719129 0.2283220077260529 tensor(182.6109)
-286.70812550199224 0.0 -286.70812550199224 0.12354206328072535 tensor(155.5527)
-287.0242918548703 0.0 -287.0242918548703 0

-286.9252326632137 0.0 -286.9252326632137 0.11408531725165307 tensor(236.2626)
-286.3742494605614 0.0 -286.3742494605614 0.2763150612122356 tensor(187.5516)
-286.8776457983935 0.0 -286.8776457983935 0.3356031422660996 tensor(196.6000)
-286.6968894792275 0.0 -286.6968894792275 0.18031224314753178 tensor(187.4603)
-286.52879833852296 0.0 -286.52879833852296 0.20499660196628036 tensor(196.2932)
-286.23646035534557 0.0 -286.23646035534557 0.11616869645490181 tensor(197.9263)
-286.1971812894324 0.0 -286.1971812894324 0.11281677575252079 tensor(253.9945)
-287.03140355543803 0.0 -287.03140355543803 0.0809041436410346 tensor(195.7730)
-286.3210987269473 0.0 -286.3210987269473 0.25428902571409967 tensor(172.7984)
-286.5775680122872 0.0 -286.5775680122872 0.23256129857538327 tensor(189.0569)
-286.53799951534137 0.0 -286.53799951534137 0.05962104168480847 tensor(224.7544)
-286.8504827948418 0.0 -286.8504827948418 0.07016775816000485 tensor(265.4737)
-286.8202145938601 0.0 -286.8202145938601 0.253

-286.87126068495024 0.0 -286.87126068495024 0.4048365863916854 tensor(187.6384)
-286.83001181783123 0.0 -286.83001181783123 0.2898011502067074 tensor(158.2594)
-286.55234327009583 0.0 -286.55234327009583 0.22451706944142108 tensor(182.6398)
-286.938263208946 0.0 -286.938263208946 0.14171217256452365 tensor(295.2348)
-286.9683559039313 0.0 -286.9683559039313 0.1747586482555117 tensor(227.1569)
-287.1036383266553 0.0 -287.1036383266553 0.32196595687691765 tensor(219.0140)
-286.3360948410062 0.0 -286.3360948410062 0.24192432746154108 tensor(239.9115)
-286.4305857291324 0.0 -286.4305857291324 0.26014798894447216 tensor(164.5104)
-287.0470088480773 0.0 -287.0470088480773 0.10195253413701251 tensor(257.5622)
-286.6679003369488 0.0 -286.6679003369488 0.23087317716387462 tensor(212.7784)
-286.8494711464491 0.0 -286.8494711464491 0.18796580405368846 tensor(256.2153)
-286.67045897516687 0.0 -286.67045897516687 0.3785439892198088 tensor(181.1894)
-287.34418615396976 0.0 -287.34418615396976 0.3343

-286.84860390138556 0.0 -286.84860390138556 0.2426777714781034 tensor(214.0343)
-287.09429707135615 0.0 -287.09429707135615 0.29540560376311575 tensor(186.0641)
-286.84173965693384 0.0 -286.84173965693384 0.23631306334163177 tensor(154.3808)
-286.9761126610591 0.0 -286.9761126610591 0.20743095891219507 tensor(158.6649)
-286.7689565388434 0.0 -286.7689565388434 0.38610577742021407 tensor(193.9017)
-286.645895121196 0.0 -286.645895121196 0.07751613582300144 tensor(180.2106)
-286.8510723349157 0.0 -286.8510723349157 0.31194353916395207 tensor(180.0549)
-287.28235879650794 0.0 -287.28235879650794 0.34750334377553804 tensor(270.9841)
-286.70169200079624 0.0 -286.70169200079624 0.33136134894152947 tensor(185.7115)
-286.8108508696748 0.0 -286.8108508696748 0.18504886480900037 tensor(194.3935)
-287.0781953992822 0.0 -287.0781953992822 0.21115048757039762 tensor(213.3158)
-286.57836898546276 0.0 -286.57836898546276 0.21001629706799327 tensor(180.1364)
-286.86788700532054 0.0 -286.86788700532054

-286.9512631343992 0.0 -286.9512631343992 0.24617413740198177 tensor(154.4555)
-287.1623977758407 0.0 -287.1623977758407 0.342355810479327 tensor(240.7198)
-286.7930173669671 0.0 -286.7930173669671 0.20959228435346472 tensor(157.4699)
-287.1452035330934 0.0 -287.1452035330934 0.16485865926838597 tensor(179.0042)
-286.9528706600418 0.0 -286.9528706600418 0.20678226512173065 tensor(178.4780)
-286.99343654530054 0.0 -286.99343654530054 0.3287748074047082 tensor(189.7015)
-286.90179416538416 0.0 -286.90179416538416 0.27938592570630255 tensor(160.7295)
-286.96891199218373 0.0 -286.96891199218373 0.3231368584825776 tensor(198.3310)
-286.9843151577437 0.0 -286.9843151577437 0.23438945491508165 tensor(161.3125)
-286.8496409542883 0.0 -286.8496409542883 0.30731494954182237 tensor(157.3821)
-286.8520106378356 0.0 -286.8520106378356 0.33597407965101855 tensor(208.6632)
-286.91932772932057 0.0 -286.91932772932057 0.25258550675454694 tensor(268.5872)
-286.8904245355953 0.0 -286.8904245355953 0.2957

In [11]:
train_force = False
flow_model = None
if train_force:
    flow_model, flow_act = flow_train(param, with_force=True, pre_model=pre_flow_model)
else:
    flow_model = pre_flow_model
flow_eval(flow_model,flow_act)
flow = flow_model['layers']
# flow.eval()

Accept rate: 0.2890625
Topological susceptibility = 1.17 +/- 0.09
... vs HMC estimate = 1.23 +/- 0.02


In [12]:
def test_force(x = None):
    model = flow_model
    layers, prior = model['layers'], model['prior']
    if x == None:
        pre_model = pre_flow_model
        pre_layers, pre_prior = pre_model['layers'], pre_model['prior']
        pre_xi = pre_prior.sample_n(1)
        x = ft_flow(pre_layers, pre_xi)
    xi = ft_flow_inv(layers, x)
    f = ft_force(param, layers, xi)
    f_s = torch.linalg.norm(f)
    print(f_s)

test_force()
test_force(torch.reshape(field,(1,)+field.shape))

tensor(20.4054)
tensor(12.3251)


In [13]:
field = run(param, field)

latsize = (8, 8)
volume = 64
beta = 2.0
trajs = 2
tau = 2
steps = 8
seed = 1331
nth = 2
nth_interop = 2
Initial configuration:  plaq: 0.662768641905577  topo: 0.0
plaq(x) 0.662768641905577  force.norm 20.998065855859643
Traj:    1  ACCEPT:  dH: -0.07537746   exp(-dH):  1.0782911    plaq:  0.64252562   topo: -2.0
plaq(x) 0.6425256187449909  force.norm 20.872768644880818
Traj:    2  ACCEPT:  dH:  0.1301976    exp(-dH):  0.87792193   plaq:  0.60711194   topo:  1.0
plaq(x) 0.6071119405479931  force.norm 19.907921445212118
Traj:    3  REJECT:  dH:  0.30832437   exp(-dH):  0.73467697   plaq:  0.60711194   topo:  1.0
plaq(x) 0.6071119405479931  force.norm 21.31516480980625
Traj:    4  ACCEPT:  dH:  0.55552569   exp(-dH):  0.57377056   plaq:  0.66056494   topo:  2.0
plaq(x) 0.6605649410770968  force.norm 17.99000516761638
Traj:    5  ACCEPT:  dH:  0.40886075   exp(-dH):  0.66440674   plaq:  0.67927934   topo:  2.0
plaq(x) 0.6792793354321754  force.norm 17.976535887639457
Traj:    6  ACCEPT:  d

In [36]:
field_run = torch.reshape(field,(1,)+field.shape)
flows = flow

print(f'plaq(field_run[0]) {action(param, field_run[0]) / (-param.beta*param.volume)}')
# field.requires_grad_(True)
x = field_run
logJ = 0.0
for layer in reversed(flows):
    x, lJ = layer.reverse(x)
    logJ += lJ

# x is the prior distribution now

x.requires_grad_(True)
    
y = x
logJy = 0.0
for layer in flows:
    y, lJ = layer.forward(y)
    logJy += lJ
    
s = action(param, y[0]) - logJy

print(logJ,logJy)


# print("eff_action", s + 136.3786)

print("original_action", action(param, y[0]) + 91)

print("eff_action", s + 56)

s.backward()

f = x.grad

x.requires_grad_(False)

print(f'plaq(x) {action(param, x[0]) / (-param.beta*param.volume)}  logJ {logJ}  force.norm {torch.linalg.norm(f)}')

print(f'plaq(y) {action(param, y[0]) / (-param.beta*param.volume)}')

print(f'plaq(x) {action(param, field_run[0]) / (-param.beta*param.volume)}  force.norm {torch.linalg.norm(force(param, field_run[0]))}')


plaq(field_run[0]) 0.6716397754361682
tensor([31.7440], grad_fn=<AddBackward0>) tensor([-31.7440], grad_fn=<AddBackward0>)
original_action tensor(5.0301, grad_fn=<AddBackward0>)
eff_action tensor([1.7741], grad_fn=<AddBackward0>)
plaq(x) -0.1024314687100247  logJ tensor([31.7440], grad_fn=<AddBackward0>)  force.norm 34.70517091343915
plaq(y) 0.67163984638713
plaq(x) 0.6716397754361682  force.norm 18.129125806140568


In [37]:
print(x.shape)
x = ft_flow_inv(flow, field_run)
# x = field_run
#for layer in reversed(flows):
#    x, lJ = layer.reverse(x)
ff = ft_force(param, flow, x)
print(torch.linalg.norm(ff))
fff = ft_force(param, flow, x)
print(torch.linalg.norm(fff))

torch.Size([1, 2, 8, 8])
tensor(34.7052)
tensor(34.7052)


In [38]:
x = ft_flow_inv(flow, field_run)
ft_action(param, flow, x)

tensor([-54.2259], grad_fn=<SubBackward0>)

In [17]:
def flattern(l):
    return [x for y in l for x in y]

def average(l):
    return sum(l) / len(l)

def sigma(l):
    avg = average(l)
    sq_avg = average([ np.square(v - avg) for v in l ])
    return sq_avg / np.sqrt(len(l) - 1)

def sub_avg(l):
    avg = average(l)
    return np.array([x - avg for x in l])

n_block = 16 # ADJUST ME

def block_list(l):
    n_block_local = n_block
    size_block = len(l) // n_block_local
    if size_block < 1:
        size_block = 1
        n_block_local = len(l)
    if n_block_local == 0:
        return []
    start = len(l) - n_block_local * size_block
    return [ l[ start + i * size_block : start + (i+1) * size_block] for i in range(n_block_local) ]

def change_sqr(l, lp):
    size = min(len(l), len(lp))
    if size == 0:
        return []
    vs = [ np.square(l[i] - lp[i]) for i in range(size) ]
    vs = list(map(average, block_list(vs)))
    avg = average(vs)
    sig = sigma(vs)
    return [ avg, sig ]

def change_sqr_vs_dt(l, dt_range = 10):
    return [ [ i ] + change_sqr(l, l[i:]) for i in range(1, dt_range + 1)]

In [40]:
ft_hmc_info_list = []
def ft_leapfrog(param, flow, x, p):
    mom_norm = torch.sum(p*p)
    info_list = []
    dt = param.dt
    x_ = x + 0.5*dt*p
    f = ft_force(param, flow, x_)
    p_ = p + (-dt)*f
    info = np.array((float(torch.linalg.norm(f)),
                     float(ft_action(param, flow, x_).detach()),
                     float(torch.sum(p*p_)/np.sqrt(mom_norm*torch.sum(p_*p_)))))
    info_list.append(info)
    for i in range(param.nstep-1):
        x_ = x_ + dt*p_
        f = ft_force(param, flow, x_)
        info = np.array((float(torch.linalg.norm(f)),
                        float(ft_action(param, flow, x_).detach()),
                        float(torch.sum(p*p_)/np.sqrt(mom_norm*torch.sum(p_*p_)))))
        info_list.append(info)
        p_ = p_ + (-dt)*f
    x_ = x_ + 0.5*dt*p_
    print(np.sqrt(average([l[0]**2 for l in info_list])),
          (info_list[0][1], info_list[-1][1]),
          info_list[-1][2])
    ft_hmc_info_list.append(info_list)
    return (x_, p_)

def ft_hmc(param, flow, field):
    x = ft_flow_inv(flow, field)
    p = torch.randn_like(x)
    act0 = ft_action(param, flow, x).detach() + 0.5*torch.sum(p*p)
    x_, p_ = ft_leapfrog(param, flow, x, p)
    xr = regularize(x_)
    act = ft_action(param, flow, xr).detach() + 0.5*torch.sum(p_*p_)
    prob = torch.rand([], dtype=torch.float64)
    dH = act-act0
    exp_mdH = torch.exp(-dH)
    acc = prob < exp_mdH
    # ADJUST ME
    newx = xr if acc else x
    # newx = xr
    newfield = ft_flow(flow, newx)
    return (float(dH), float(exp_mdH), acc, newfield)

In [110]:
def ft_run(param, flow, field = None):
    if field == None:
        field = param.initializer()
    ft_hmc_info_list = []
    with open(param.uniquestr(), "w") as O:
        params = param.summary()
        O.write(params)
        put(params)
        plaq, topo = (action(param, field) / (-param.beta*param.volume), topocharge(field))
        status = f"Initial configuration:  plaq: {plaq}  topo: {topo} {field.shape}\n"
        O.write(status)
        put(status)
        ts = []
        for n in range(param.nrun):
            t = -timer()
            for i in range(param.ntraj):
                field_run = torch.reshape(field,(1,)+field.shape)
                dH, exp_mdH, acc, field_run = ft_hmc(param, flow, field_run)
                field = field_run[0]
                plaq = action(param, field) / (-param.beta*param.volume)
                topo = topocharge(field)
                ifacc = "ACCEPT" if acc else "REJECT"
                status = f"Traj: {n*param.ntraj+i+1:4}  {ifacc}:  dH: {dH:< 12.8}  exp(-dH): {exp_mdH:< 12.8}  plaq: {plaq:< 12.8}  topo: {topo:< 3.3}\n"
                O.write(status)
                if (i+1) % (param.ntraj//param.nprint) == 0:
                    put(status)
            t += timer()
            ts.append(t)
        print("Run times: ", ts)
        print("Per trajectory: ", [t/param.ntraj for t in ts])
    return field

In [112]:
param = Param(
    beta = 2.0,
    lat = (8, 8),
    tau = 0.5, # 0.3
    nstep = 64, # 3
    # ADJUST ME
    ntraj = 4, # 2**16 # 2**10 # 2**15
    nprint = 4,
    #
    seed = 1331)

# field = ft_run(param, pre_flow)
# field = ft_run(param, pre_flow, field)
field = ft_run(param, flow, field)

latsize = (8, 8)
volume = 64
beta = 2.0
trajs = 4
tau = 0.5
steps = 64
seed = 1331
nth = 2
nth_interop = 2
Initial configuration:  plaq: 0.7887135679736341  topo: 1.0 torch.Size([2, 8, 8])


KeyboardInterrupt: 

In [109]:
param = Param(
    beta = 2.0,
    lat = (8, 8),
    tau = 0.5, # 0.3
    nstep = 64, # 3
    # ADJUST ME
    ntraj = 4, # 2**16 # 2**10 # 2**15
    nprint = 4,
    #
    seed = 1331)

# field = ft_run(param, pre_flow)
# field = ft_run(param, pre_flow, field)
field = ft_run(param, flow, field)

latsize = (8, 8)
volume = 64
beta = 2.0
trajs = 4
tau = 0.5
steps = 64
seed = 1331
nth = 2
nth_interop = 2
Initial configuration:  plaq: 0.7887135679736341  topo: 1.0


KeyboardInterrupt: 

In [43]:
action_list = np.array([l[1] for l in flattern(ft_hmc_info_list)])
action_list = sub_avg(action_list)
np.sqrt(average(action_list**2))

0.8744031477616621

In [44]:
force_list = np.array([l[0] for l in flattern(ft_hmc_info_list)])
np.sqrt(average(force_list**2))

32.58414880207859

In [45]:
print(np.array(force_list[0:300]))

[ 36.09316701  26.38977078  68.00603671  31.22982216  29.12227097
  21.62493451  18.86294066  16.22557307  13.16750373  10.03210621
   7.26475395   5.25456284   4.48932086   5.28910149   7.1604243
   9.41633164  11.56022469  13.34897174  15.0347588   17.06472245
  13.63004187  13.67601611  23.44102517  15.40261377  13.8023178
  13.10502926   9.80238718   9.96926622  12.14492081  11.3295407
   9.74145174   8.83835725   8.45431986   8.16819299   7.74117561
   7.01310237   5.88640763   5.09706541   7.34167767  11.13599049
  11.48665577   8.42736975   6.56570125   6.74110212   7.24219414
   7.34891711   6.94702375   6.30927502   5.95707362   6.37242431
   8.58184857  14.59622813  21.46877935  21.35009509  14.97622465
  10.38055866   9.55276204  10.11013628  11.13003905  12.4687411
  14.04994638  15.84468009  17.9148159   20.32374896  21.45948374
  20.98446039  20.24634092  19.33385882  18.35541384  17.40966759
  16.56214508  15.83911804  15.23833978  14.74681446  14.354167
  14.05499849  1

In [114]:
param_new = Param(
    beta = 2.0,
    lat = (12, 12),
    tau = 0.5, # 0.3
    nstep = 64, # 3
    # ADJUST ME
    ntraj = 4, # 2**16 # 2**10 # 2**15
    nprint = 4,
    #
    seed = 1331)

field_new = run(param_new)

latsize = (12, 12)
volume = 144
beta = 2.0
trajs = 4
tau = 0.5
steps = 64
seed = 1331
nth = 2
nth_interop = 2
Initial configuration:  plaq: 1.0  topo: 0.0 torch.Size([2, 12, 12])
plaq(x) 1.0  force.norm 0.39715422715538007
Traj:    1  ACCEPT:  dH: -0.0062864964  exp(-dH):  1.0063063    plaq:  0.81235594   topo:  0.0
plaq(x) 0.812355938734624  force.norm 27.202019368757615
Traj:    2  ACCEPT:  dH: -0.0015405631  exp(-dH):  1.0015418    plaq:  0.7492163    topo:  0.0
plaq(x) 0.7492163023596704  force.norm 29.485665083133863
Traj:    3  ACCEPT:  dH:  0.00037426607  exp(-dH):  0.9996258    plaq:  0.72375702   topo:  0.0
plaq(x) 0.7237570215411304  force.norm 28.303420580973683
Traj:    4  ACCEPT:  dH: -0.0013935697  exp(-dH):  1.0013945    plaq:  0.67309378   topo:  0.0
plaq(x) 0.6730937834511482  force.norm 29.926790016437316
Traj:    5  ACCEPT:  dH:  0.00020590437  exp(-dH):  0.99979412   plaq:  0.67381348   topo:  0.0
plaq(x) 0.6738134751318049  force.norm 29.254445461455482
Traj:    6 

In [115]:
def get_nets(layers):
    nets = []
    for l in layers:
        nets.append(l.plaq_coupling.net)
    return nets

In [116]:
def make_u1_equiv_layers_net(*, lattice_shape, nets):
    n_layers = len(nets)
    layers = []
    for i in range(n_layers):
        # periodically loop through all arrangements of maskings
        mu = i % 2
        off = (i//2) % 4
        net = nets[i]
        plaq_coupling = NCPPlaqCouplingLayer(
            net, mask_shape=lattice_shape, mask_mu=mu, mask_off=off)
        link_coupling = GaugeEquivCouplingLayer(
            lattice_shape=lattice_shape, mask_mu=mu, mask_off=off, 
            plaq_coupling=plaq_coupling)
        layers.append(link_coupling)
    return torch.nn.ModuleList(layers)

In [117]:
len(get_nets(flow))

24

In [125]:
type(get_nets(flow)[0])

torch.nn.modules.container.Sequential

In [128]:
param.beta

2.0

In [129]:
torch.save(flow, f"flow_8x8_beta={param.beta}.dat")

In [130]:
flow_load = torch.load(f"flow_8x8_beta={param.beta}.dat")

In [131]:
ft_run(param, flow_load)

latsize = (8, 8)
volume = 64
beta = 2.0
trajs = 4
tau = 0.5
steps = 64
seed = 1331
nth = 2
nth_interop = 2
Initial configuration:  plaq: 1.0  topo: 0.0 torch.Size([2, 8, 8])
2.130422682286812 (-53.222385355919386, -53.429393766012076) 0.9978707399156546
Traj:    1  ACCEPT:  dH: -0.0042142763  exp(-dH):  1.0042232    plaq:  0.90614763   topo:  1.0
4.029733486080523 (-53.417886748435585, -54.24536203857387) 0.9946731949446221
Traj:    2  ACCEPT:  dH:  1.1748347e-05  exp(-dH):  0.99998825   plaq:  0.87431725   topo:  0.0
18.183895995414918 (-54.23038681455611, -54.38308753681161) 0.9545260703723759
Traj:    3  ACCEPT:  dH:  0.0027780899  exp(-dH):  0.99722577   plaq:  0.80137963   topo:  0.0
16.457871831575794 (-54.31892098117961, -53.913041180964) 0.927469245598059
Traj:    4  ACCEPT:  dH: -0.0010801517  exp(-dH):  1.0010807    plaq:  0.77648579   topo: -2.0
23.428317208817752 (-53.88039348814588, -55.737052236478576) 0.8380530816381841
Traj:    5  ACCEPT:  dH: -0.011471572  exp(-dH):  1

tensor([[[0.6534, 5.4021, 0.4575, 4.8465, 2.8914, 1.5222, 4.7147, 0.3549],
         [1.6744, 1.3655, 4.9173, 5.0352, 1.6899, 1.1627, 0.1941, 1.0700],
         [4.2327, 6.2283, 0.1999, 0.8074, 0.2348, 3.1774, 3.9142, 3.4625],
         [2.6332, 3.2829, 5.9404, 1.4562, 1.1028, 2.6821, 5.4223, 3.7265],
         [1.5549, 4.5315, 3.9697, 3.7680, 4.5322, 5.7528, 0.5089, 4.0359],
         [1.0642, 4.0047, 3.9317, 0.2294, 1.9237, 1.4102, 4.9743, 2.5347],
         [4.9772, 2.9403, 0.8832, 2.8796, 6.2385, 0.0933, 2.3724, 5.5047],
         [0.5177, 2.0975, 2.1847, 4.3087, 2.8215, 0.7616, 5.9679, 5.9345]],

        [[2.5493, 5.9101, 6.0390, 1.0026, 5.9397, 4.5870, 0.4594, 0.5701],
         [0.9451, 1.3213, 4.4304, 6.0886, 4.6240, 1.1671, 1.5940, 0.6317],
         [0.2708, 5.4701, 6.0213, 3.3811, 3.2066, 5.7253, 1.9331, 5.7300],
         [1.8646, 5.7644, 5.8269, 1.3982, 6.1795, 0.2834, 1.7275, 0.1077],
         [1.7839, 0.3296, 1.7925, 0.6204, 0.3223, 0.9301, 5.8468, 4.2750],
         [5.2899, 0.141

In [118]:
flow_new = make_u1_equiv_layers_net(lattice_shape = param_new.lat, nets = get_nets(flow))

In [119]:
field_new = ft_run(param_new, flow_new)

latsize = (12, 12)
volume = 144
beta = 2.0
trajs = 4
tau = 0.5
steps = 64
seed = 1331
nth = 2
nth_interop = 2
Initial configuration:  plaq: 1.0  topo: 0.0 torch.Size([2, 12, 12])
1.9879761523473762 (-119.74963665923889, -118.76456592618388) 0.9988032924479914
Traj:    1  ACCEPT:  dH:  0.0011222239  exp(-dH):  0.99887841   plaq:  0.93142052   topo:  0.0
12.42193092625858 (-118.75965918089585, -118.93633117431767) 0.9910858034285454
Traj:    2  ACCEPT:  dH: -0.0095857972  exp(-dH):  1.0096319    plaq:  0.844902     topo:  0.0
15.478099801405712 (-118.87205347789688, -119.94980202022144) 0.9819081817417009
Traj:    3  ACCEPT:  dH:  0.0056008684  exp(-dH):  0.99441479   plaq:  0.80854077   topo: -1.0
17.494684446804182 (-119.8917686966347, -117.0291676196318) 0.9750739933170334
Traj:    4  ACCEPT:  dH: -0.0034243272  exp(-dH):  1.0034302    plaq:  0.71301701   topo:  1.0
25.98443996296935 (-117.08010794260822, -116.4797207140743) 0.9456847457252119
Traj:    5  ACCEPT:  dH:  0.013018336  ex

In [120]:
field_new = ft_run(param_new, flow_new, field_new)

latsize = (12, 12)
volume = 144
beta = 2.0
trajs = 4
tau = 0.5
steps = 64
seed = 1331
nth = 2
nth_interop = 2
Initial configuration:  plaq: 0.7165461986580746  topo: 1.0 torch.Size([2, 12, 12])
33.205227499877566 (-119.69422351167141, -120.6127662698634) 0.9372124560816845
Traj:    1  ACCEPT:  dH: -0.11566231   exp(-dH):  1.1226167    plaq:  0.70306027   topo:  1.0
44.41642108291006 (-120.64624274028078, -121.32630719456144) 0.8803180973693999
Traj:    2  ACCEPT:  dH: -0.03176109   exp(-dH):  1.0322709    plaq:  0.69111901   topo:  0.0
37.033858167006905 (-121.26552062339381, -120.25376386065368) 0.9415684068308903
Traj:    3  ACCEPT:  dH: -0.024179267  exp(-dH):  1.024474     plaq:  0.69309801   topo: -1.0
32.78631415560145 (-120.32965204440877, -124.04080016816137) 0.9446849064872301
Traj:    4  ACCEPT:  dH: -0.005851262  exp(-dH):  1.0058684    plaq:  0.69533788   topo:  0.0
94.55484287198264 (-124.03655090387468, -119.75372667268061) 0.8925964152188036
Traj:    5  REJECT:  dH:  14.

In [121]:
action(param, field)

tensor(-100.9553)

In [122]:
action(param_new, field_new)

tensor(-200.1450)

In [29]:
def print_topo_change_sqr(topo_history):
    size_hist = len(topo_history)
    drop_len = size_hist // 3 # ADJUST ME
    dt = change_sqr_vs_dt(topo_history[drop_len:])
    for l in dt:
        print(l)

def non_empty(str):
    return not(str == "" or str == "\n")
        
fns = {
    "v6/out_b5.0_l16.16_n128_t1.0_s32.out.topo_history":"Small volume, original",
    "v6/out_b5.0_l16.16_n128_t1.0_s64_ft.out.topo_history":"Small volume, Field Transformation",
    "v6/out_b5.0_l32.32_n128_t1.0_s32.out.topo_history":"Large volume, original",
    "v6/out_b5.0_l32.32_n128_t1.0_s64_ft.out.topo_history":"Large volume, same field Transformation",
}

for (fn, description) in fns.items():
    print(description)
    print(fn)
    with open(fn, "r") as f:
        lines = f.readlines()
    lines = list(filter(non_empty, lines))
    topo_history = list(map(float, lines))
    print_topo_change_sqr(topo_history)

Small volume, original
v6/out_b5.0_l16.16_n128_t1.0_s32.out.topo_history
[1, 0.01488095238095238, 0.0001989734216270108]
[2, 0.02976190476190476, 0.0007227080601624759]
[3, 0.04464285714285714, 0.0012784614102241273]
[4, 0.05952380952380952, 0.002085790350848665]
[5, 0.0744047619047619, 0.0031446948820360895]
[6, 0.08928571428571429, 0.004089246872058566]
[7, 0.109375, 0.00711811978277574]
[8, 0.125, 0.00903696114115064]
[9, 0.140625, 0.010749041669845194]
[10, 0.15625, 0.01281917144018132]
Small volume, Field Transformation
v6/out_b5.0_l16.16_n128_t1.0_s64_ft.out.topo_history
[1, 0.13690476190476192, 0.0025523487188016565]
[2, 0.24702380952380948, 0.006309973221481873]
[3, 0.36011904761904756, 0.01309794006503323]
[4, 0.47321428571428564, 0.025942017488680285]
[5, 0.5744047619047619, 0.03457792139745722]
[6, 0.6785714285714286, 0.044240711125895386]
[7, 0.765625, 0.05573212949298342]
[8, 0.8250000000000001, 0.059385744641847066]
[9, 0.8968750000000001, 0.07830436055759851]
[10, 1.0, 0